In [17]:
import torch
import torch.nn as nn
import math

# Self Attention V1

$$
\text{softmax}(\frac{QK^T}{\sqrt{d_k}})V
$$

In [18]:

class SelfAttV1(nn.Module):
    def __init__(self, hidden_dim: int):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, hidden_dim)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, X: torch.Tensor, use_einsum=False):
        # X's shape: (b, s, h)
        Q = self.q_proj(X) # (b, s, h)
        K = self.q_proj(X) # (b, s, h)
        V = self.q_proj(X) # (b, s, h)

        if use_einsum:
            att_value = torch.einsum('bih, bjh -> bij', Q, K) / math.sqrt(self.hidden_dim)
        else:
            att_value = Q @ K.transpose(-1,-2) / math.sqrt(self.hidden_dim) # (b, s, s)

        att_weight = torch.softmax(att_value, dim=-1)

        output = att_weight @ V # (b, s, h)

        return output


In [19]:
X = torch.randn((4, 2, 2))
net = SelfAttV1(hidden_dim = X.shape[-1])
output = net(X)
print(output.shape)
print(output)

torch.Size([4, 2, 2])

tensor([[[-0.2657, -0.9009],
         [-0.3553, -1.0871]],

        [[-0.6629, -1.6758],
         [-0.5504, -1.4616]],

        [[-1.0018, -2.3841],
         [-0.9101, -2.2020]],

        [[-0.2875, -0.9899],
         [-0.2876, -0.9899]]], grad_fn=<UnsafeViewBackward0>)

In [20]:
output = net(X, use_einsum=True)
print(output.shape)
print(output)

torch.Size([4, 2, 2])

tensor([[[-0.2657, -0.9009],
         [-0.3553, -1.0871]],

        [[-0.6629, -1.6758],
         [-0.5504, -1.4616]],

        [[-1.0018, -2.3841],
         [-0.9101, -2.2020]],

        [[-0.2875, -0.9899],
         [-0.2876, -0.9899]]], grad_fn=<UnsafeViewBackward0>)

# Self Attention V2
- 合并qkv的矩阵乘法，提高效率. 虽然现在大模型时代，q k v 的投影矩阵都是分开写的，这是因为现在的模型很大，本身可能会做 张量并行，流水线并行等方式，所以分开写问题也不大（分开写很清晰）。
- 加入attention mask
- output也加一个线性层

In [21]:
class SelfAttV2(nn.Module):
    def __init__(self, hidden_dim: int):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.qkv_proj = nn.Linear(hidden_dim, hidden_dim * 3)
        self.o_proj = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, X: torch.Tensor, att_mask: torch.Tensor):
        # X's shape: (b, s, h)
        # att_mask's shape: (b, s, s)
        QKV = self.qkv_proj(X) # (b, s, h * 3)

        Q, K, V = torch.split(QKV, self.hidden_dim, dim=-1)
        att_value = torch.einsum('bih, bjh -> bij', Q, K) / math.sqrt(self.hidden_dim)
        if att_mask is not None:
            att_value.masked_fill_(att_mask == 0, float("-inf"))
        att_weight = torch.softmax(att_value, dim=-1)
        output = self.o_proj(att_weight @ V)
        return output

In [22]:
X = torch.randn((3, 4, 2))
net = SelfAttV2(hidden_dim = X.shape[-1])
att_mask = torch.Tensor([
    [1, 1, 1, 1],
    [1, 1, 1, 0],
    [1, 0, 0, 0],
])
att_mask = att_mask.unsqueeze(dim=1).repeat(1, 4, 1)
print(att_mask.shape)
output = net(X, att_mask)
print(output.shape)
print(output)



torch.Size([3, 4, 4])

torch.Size([3, 4, 2])

tensor([[[-0.3721, -0.4338],
         [-0.3884, -0.4227],
         [-0.3112, -0.4751],
         [-0.3422, -0.4541]],

        [[-0.1920, -0.5326],
         [-0.1491, -0.5620],
         [-0.1391, -0.5688],
         [-0.2426, -0.4980]],

        [[-0.3832, -0.4232],
         [-0.3832, -0.4232],
         [-0.3832, -0.4232],
         [-0.3832, -0.4232]]], grad_fn=<ViewBackward0>)


# Self Attention V3
- 加入dropout。很奇怪的一点是，在BERT里面，这里的dropout用在了att_weight上面，不是output上面

In [23]:
class SelfAttV3(nn.Module):
    def __init__(self, hidden_dim: int, att_drop_p: float = 0.1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.qkv_proj = nn.Linear(hidden_dim, hidden_dim * 3)
        self.o_proj = nn.Linear(hidden_dim, hidden_dim)
        self.att_drop = nn.Dropout(att_drop_p)

    def forward(self, X: torch.Tensor, att_mask: torch.Tensor):
        QKV = self.qkv_proj(X)
        Q, K, V = torch.split(QKV, self.hidden_dim, dim=-1)
        att_value = torch.einsum('bih, bjh -> bij', Q, K)
        if att_mask is not None:
            att_value.masked_fill_(att_mask ==0, float('-inf'))
        att_weight = self.att_drop(torch.softmax(att_value, dim=-1))
        output = self.o_proj(att_weight @ V)
        return output

X = torch.randn((3, 4, 2))
net = SelfAttV3(hidden_dim = X.shape[-1])
att_mask = torch.Tensor([
    [1, 1, 1, 1],
    [1, 1, 1, 0],
    [1, 0, 0, 0],
])
att_mask = att_mask.unsqueeze(dim=1).repeat(1, 4, 1)
output = net(X, att_mask)
print(output.shape)
print(output)

torch.Size([3, 4, 2])

tensor([[[ 0.2652,  0.8264],
         [ 0.2216,  0.7890],
         [ 0.3596,  1.0809],
         [ 0.2110,  0.8993]],

        [[ 0.3742,  1.1257],
         [ 0.3779,  1.1462],
         [ 0.3204,  1.0678],
         [ 0.3408,  1.0699]],

        [[-0.4950, -0.1857],
         [-0.4950, -0.1857],
         [-0.4950, -0.1857],
         [-0.4950, -0.1857]]], grad_fn=<ViewBackward0>)